In [1]:
import xml.etree.ElementTree as ET
from cvkg.flextglutils import *

In [2]:
tree = ET.parse('c:/tmp/vk.xml')
registry = tree.getroot()

tags_node = registry.find('tags')
types_node = registry.find('types')
enum_nodes = registry.findall('enums')
feature_nodes = registry.findall('feature')
extension_nodes = registry.find('extensions')
commands_node = registry.find('commands')

In [3]:
types_node.attrib

{'comment': 'Vulkan type definitions'}

In [4]:
VkGeometryTrianglesNV = types_node[1000]

In [5]:
def node_text(node: ET.Element):
    if node.text is not None:
        return node.text
    return ''.join([node_text(c) for c in node])

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
from cvkg import SpecParser

parser = SpecParser("c:/tmp/vk.v1.2.203.xml")

Starting to parse 'c:/tmp/vk.v1.2.203.xml':
1508 Types found
510 commands found
215 Enums found
Parsing feature level VK_VERSION_1_0
  Header boilerplate
  Fundamental types used by many commands and structures
  These types are part of the API, though not directly used in API commands or data structures
  API version macros
  API constants
   <enum> VK_ATTACHMENT_UNUSED
    -->not found
   <enum> VK_FALSE
    -->not found
   <enum> VK_LOD_CLAMP_NONE
    -->not found
   <enum> VK_QUEUE_FAMILY_IGNORED
    -->not found
   <enum> VK_REMAINING_ARRAY_LAYERS
    -->not found
   <enum> VK_REMAINING_MIP_LEVELS
    -->not found
   <enum> VK_SUBPASS_EXTERNAL
    -->not found
   <enum> VK_TRUE
    -->not found
   <enum> VK_WHOLE_SIZE
    -->not found
  Device initialization
  Device commands
  Extension discovery commands
  Layer discovery commands
  Queue commands
  Memory commands
  Memory management API commands
  Sparse resource memory management API commands
  Fence commands
  Queue semaphor

In [10]:
parser.feature_levels['VK_VERSION_1_1'].enums

{'VkStructureType': [enum] VkStructureType
   - VK_STRUCTURE_TYPE_APPLICATION_INFO: 0
   - VK_STRUCTURE_TYPE_INSTANCE_CREATE_INFO: 1
   - VK_STRUCTURE_TYPE_DEVICE_QUEUE_CREATE_INFO: 2
   - VK_STRUCTURE_TYPE_DEVICE_CREATE_INFO: 3
   - VK_STRUCTURE_TYPE_SUBMIT_INFO: 4
   - VK_STRUCTURE_TYPE_MEMORY_ALLOCATE_INFO: 5
   - VK_STRUCTURE_TYPE_MAPPED_MEMORY_RANGE: 6
   - VK_STRUCTURE_TYPE_BIND_SPARSE_INFO: 7
   - VK_STRUCTURE_TYPE_FENCE_CREATE_INFO: 8
   - VK_STRUCTURE_TYPE_SEMAPHORE_CREATE_INFO: 9
   - VK_STRUCTURE_TYPE_EVENT_CREATE_INFO: 10
   - VK_STRUCTURE_TYPE_QUERY_POOL_CREATE_INFO: 11
   - VK_STRUCTURE_TYPE_BUFFER_CREATE_INFO: 12
   - VK_STRUCTURE_TYPE_BUFFER_VIEW_CREATE_INFO: 13
   - VK_STRUCTURE_TYPE_IMAGE_CREATE_INFO: 14
   - VK_STRUCTURE_TYPE_IMAGE_VIEW_CREATE_INFO: 15
   - VK_STRUCTURE_TYPE_SHADER_MODULE_CREATE_INFO: 16
   - VK_STRUCTURE_TYPE_PIPELINE_CACHE_CREATE_INFO: 17
   - VK_STRUCTURE_TYPE_PIPELINE_SHADER_STAGE_CREATE_INFO: 18
   - VK_STRUCTURE_TYPE_PIPELINE_VERTEX_INPUT_STATE

In [16]:
feature_1_0 = parser.features

In [17]:
parser.features.enums['abd'] = 1

In [20]:
import copy

In [65]:
print(parser.types['VkResult'])
print(parser.types['VkFramebufferCreateFlagBits'])
print(parser.types['VkImageFormatListCreateInfo'])
print(parser.types['VkColorSpaceKHR'])
print(parser.types['VkDriverIdKHR'])
print(parser.types['VkSamplerCreateFlags'])


[enum] VkResult
[enum] VkFramebufferCreateFlagBits
[struct] VkImageFormatListCreateInfo
  -  sType: VkStructureType
  - ?pNext: const void*
  - ?viewFormatCount: uint32_t
  -  pViewFormats: const VkFormat*
[enum] VkColorSpaceKHR
[enum] VkDriverIdKHR --> VkDriverId
[bitmask] VkSamplerCreateFlags --> VkFlags


In [66]:
print(parser.commands['vkCreateInstance'])
print(parser.commands['vkCmdDispatch'])
print(parser.commands['vkCmdSetDiscardRectangleEXT'])
print(parser.commands['vkGetImageMemoryRequirements2'])

[command] vkCreateInstance(...) -> VkResult
  -  pCreateInfo: const VkInstanceCreateInfo*
  - ?pAllocator: const VkAllocationCallbacks*
  -  pInstance: VkInstance
[command] vkCmdDispatch(...) -> void
  -  commandBuffer: VkCommandBuffer
  -  groupCountX: uint32_t
  -  groupCountY: uint32_t
  -  groupCountZ: uint32_t
[command] vkCmdSetDiscardRectangleEXT(...) -> void
  -  commandBuffer: VkCommandBuffer
  -  firstDiscardRectangle: uint32_t
  -  discardRectangleCount: uint32_t
  -  pDiscardRectangles: const VkRect2D*
[command] vkGetImageMemoryRequirements2(...) -> void
  -  device: VkDevice
  -  pInfo: const VkImageMemoryRequirementsInfo2*
  -  pMemoryRequirements: VkMemoryRequirements2


In [67]:
print(parser.commands['vkCreateImage'])
print(parser.types['VkImageCreateInfo'])

[command] vkCreateImage(...) -> VkResult
  -  device: VkDevice
  -  pCreateInfo: const VkImageCreateInfo*
  - ?pAllocator: const VkAllocationCallbacks*
  -  pImage: VkImage
[struct] VkImageCreateInfo
  -  sType: VkStructureType
  - ?pNext: const void*
  - ?flags: VkImageCreateFlags
  -  imageType: VkImageType
  -  format: VkFormat
  -  extent: VkExtent3D
  -  mipLevels: uint32_t
  -  arrayLayers: uint32_t
  -  samples: VkSampleCountFlagBits
  -  tiling: VkImageTiling
  -  usage: VkImageUsageFlags
  -  sharingMode: VkSharingMode
  - ?queueFamilyIndexCount: uint32_t
  -  pQueueFamilyIndices: const uint32_t*
  -  initialLayout: VkImageLayout


# Consistency check of the parsed data

In [68]:
from cvkg.Types import *

In [69]:
# list enums that are missing a definition of their values
# alias enums are expected to not have values.
undefined_enums = [t for t in parser.types.values() if t.category == 'enum' and not t.alias and t.name not in parser.enums.keys()]
display(undefined_enums)
def unresolved_types(paramsOrMembers: List[Tuple[Parameter,Member]])->List[str]:
    unresolved = []
    for p in paramsOrMembers:
        if p.type not in parser.types:
            unresolved.append(p.type)
    return unresolved
# list types with undefined member types
types_with_undefined_members = [t for t in parser.types.values() if len(unresolved_types(t.members)) > 0]
display(types_with_undefined_members)

[[enum] VkQueryPoolCreateFlagBits,
 [enum] VkInstanceCreateFlagBits,
 [enum] VkDeviceCreateFlagBits]

[]

In [70]:
# list types with undefined member types

def unresolved_types(paramsOrMembers: List[Tuple[Parameter,Member]])->List[str]:
    unresolved = []
    for p in paramsOrMembers:
        if p.type not in parser.types:
            unresolved.append(p.type)
    return unresolved

types_with_undefined_members = [t for t in parser.types.values() if len(unresolved_types(t.members)) > 0]
display(types_with_undefined_members)

[]

In [71]:
# list commands with undefined parameter types
commands_with_undefined_parameters = [c for c in parser.commands.values() if not c.alias and len(unresolved_types(c.params)) > 0]
display(commands_with_undefined_parameters)

[]

In [72]:
# do all `vkCmd` commands have a commandBuffer as a first argument?
vkCmdCommands = [c for c in parser.commands.values() if c.name.startswith('vkCmd') and not c.alias]
print(f"{len(vkCmdCommands)} vkCmd commands")
[c for c in vkCmdCommands if len(c.params) == 0 or c.params[0].type != 'VkCommandBuffer']

144 vkCmd commands


[]

In [75]:
parser.enums['VkFullScreenExclusiveEXT']

[enum] VkFullScreenExclusiveEXT
  - VK_FULL_SCREEN_EXCLUSIVE_DEFAULT_EXT: 0
  - VK_FULL_SCREEN_EXCLUSIVE_ALLOWED_EXT: 1
  - VK_FULL_SCREEN_EXCLUSIVE_DISALLOWED_EXT: 2
  - VK_FULL_SCREEN_EXCLUSIVE_APPLICATION_CONTROLLED_EXT: 3

In [78]:
[c.name for c in parser.commands.values() if not c.alias and c.name.startswith == 'VkVideo']

[]

# Velocity Template generation playground

In [16]:
# load the data model
from cvkg import SpecParser

parser = SpecParser("c:/tmp/vk.v1.2.203.xml")

Starting to parse 'c:/tmp/vk.v1.2.203.xml':
1508 Types found
510 commands found
215 Enums found
===== Summary =====
Types:     1508
Commands:  510
Enums:     215

=== Parsed 'c:/tmp/vk.v1.2.203.xml' in  109.0 ms ===


In [30]:
create_infos = [c for c in parser.types.values() if not c.alias and "CreateInfo" in c.name]

In [63]:
from airspeed import Template,CachingFileLoader

In [62]:
env = {'create_info': create_infos[0]}
t = Template("""
struct ${create_info.name} {
#foreach ( $member in $create_info.members )
    #if( $member.values )${member.type} ${member.name}{$member.values};#else${member.type} ${member.name};#end
#end

};
""")
print(t.merge(env))


struct VkDeviceQueueCreateInfo {

    VkStructureType sType{VK_STRUCTURE_TYPE_DEVICE_QUEUE_CREATE_INFO};
    void pNext;
    VkDeviceQueueCreateFlags flags;
    uint32_t queueFamilyIndex;
    uint32_t queueCount;
    float pQueuePriorities;
};



In [64]:
loader = CachingFileLoader('./cvkg/templates/')
template = loader.load_template('CreateInfo.tpl.h')
print(template.merge(env))

struct VkDeviceQueueCreateInfo {

    VkStructureType sType{VK_STRUCTURE_TYPE_DEVICE_QUEUE_CREATE_INFO};
    void pNext;
    VkDeviceQueueCreateFlags flags;
    uint32_t queueFamilyIndex;
    uint32_t queueCount;
    float pQueuePriorities;
};


In [67]:
?CachingFileLoader

Init signature: CachingFileLoader(basedir, debugging=False)
Docstring:      <no docstring>
File:           c:\dev\cory\coryvkgen\.cvkg-env\lib\site-packages\airspeed\__init__.py
Type:           type
Subclasses:     


In [73]:
env = {'create_infos': create_infos[0:3]}
t = Template("""
#foreach ( $create_info in $create_infos)
#parse ( "CreateInfo.tpl.h")
#end
""")
print(t.merge(env, loader))



struct VkDeviceQueueCreateInfo { \

    VkStructureType sType{VK_STRUCTURE_TYPE_DEVICE_QUEUE_CREATE_INFO};
    void pNext;
    VkDeviceQueueCreateFlags flags;
    uint32_t queueFamilyIndex;
    uint32_t queueCount;
    float pQueuePriorities;
};

struct VkDeviceCreateInfo { \

    VkStructureType sType{VK_STRUCTURE_TYPE_DEVICE_CREATE_INFO};
    void pNext;
    VkDeviceCreateFlags flags;
    uint32_t queueCreateInfoCount;
    VkDeviceQueueCreateInfo pQueueCreateInfos;
    uint32_t enabledLayerCount;
    char ppEnabledLayerNames;
    uint32_t enabledExtensionCount;
    char ppEnabledExtensionNames;
    VkPhysicalDeviceFeatures pEnabledFeatures;
};

struct VkInstanceCreateInfo { \

    VkStructureType sType{VK_STRUCTURE_TYPE_INSTANCE_CREATE_INFO};
    void pNext;
    VkInstanceCreateFlags flags;
    VkApplicationInfo pApplicationInfo;
    uint32_t enabledLayerCount;
    char ppEnabledLayerNames;
    uint32_t enabledExtensionCount;
    char ppEnabledExtensionNames;
};

